# Capao Urban Rate

This notebook aims to estimate the current urban growth rate for the place: Vale do Capão, Palmeiras, BA, Brasil. By using Google Earth satellite images and analyzing RGB pixel data, we can determine the growth rate in areas featuring houses, roads, construction sites, or where forests have been cleared for humans uses. For this case study, we have chosen COPERNICUS satellite images.

In [ ]:
import ee
import geemap as geemap
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt

from utils.contants import PROJECT
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split


ee.Authenticate()
ee.Initialize(project=PROJECT)
geemap.ee_initialize()

geo_point = ee.Geometry.Point([-41.501150593949305,-12.609558240448216])

geo_place = ee.FeatureCollection([
  ee.Feature(
    ee.Geometry.Polygon([
      [-41.51551178850023,-12.571410698153873],
      [-41.51392392076341,-12.575557425201367],
      [-41.51349476732103,-12.596499467766318],
      [-41.516777307708075,-12.605673846986134],
      [-41.513816148955634,-12.613023835365905],
      [-41.510018140990546,-12.61471995657374],
      [-41.5042460271905,-12.622069685275841],
      [-41.49767997952204,-12.633585929553394],
      [-41.495040685851386,-12.64577164489336],
      [-41.49165037365656,-12.65577940923046],
      [-41.48326410447515,-12.654326569148258],
      [-41.482418542332205,-12.629461766530518],
      [-41.484178071445974,-12.62054181330977],
      [-41.486409669346365,-12.610658309543654],
      [-41.4919457487531,-12.603664243777065],
      [-41.4923984315149,-12.595572883050577],
      [-41.48810689709107,-12.581960700605178],
      [-41.48832147381226,-12.568473457194342],
      [-41.512010743831794,-12.568347796317699],
      [-41.51551178850023,-12.571410698153873]
    ])
  )
])

image = (
  ee.Image(
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(geo_place)
      .filterDate('2023-01-01', '2023-12-01')
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
  )
)
# B2 - Blue | B3 - Green |  B4 - Red
bands = ['B2', 'B3', 'B4']


# geo_place = image.select(bands).sampleRegions(
#   collection=polygon,
#   properties=['landcover'],
#   scale=10,
#   geometries=True
# )

## Map view

In [41]:
map = geemap.Map(center=[-12.609558240448216,-41.501150593949305], zoom=15)

# Adjust this value to increase or decrease the contrast
vis_params = {"bands": bands,  min: 0, max: 2000, "gamma": 3.5}
map.add_layer(image, vis_params, 'Polygon')

# Display the map
map

Map(center=[-12.609558240448216, -41.501150593949305], controls=(WidgetControl(options=['position', 'transpare…

## Pre processing

In [ ]:
# df = pd.read_csv('./data/dataset_bands_3.csv')
df = pd.read_csv('./data/dataset_bands_6.csv')

# dataset = df[['system:index', 'B2', 'B3', 'B4']].copy()
dataset = df[['system:index', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']].copy()

dataset = dataset.replace(regex=r'1_\d+_\d+', value=0) # Urban Class 0
dataset = dataset.replace(regex=r'2_\d+_\d+', value=1) # Urban Class 1
dataset.rename(columns={'system:index': 'class'}, inplace=True)
dataset[110:130]


## Data Clusters

In [ ]:
# Normalizing the data to be in the range (0, 1)
scaler = MinMaxScaler()
dataset_normalized = scaler.fit_transform(dataset)

# PCA to reduce dimensions from 3 to 2
pca = PCA(n_components=2)
data_reduced = pca.fit_transform(dataset_normalized)

# K-means clustering
kmeans = KMeans(n_clusters=2)
clusters = kmeans.fit_predict(data_reduced)

# Visualize clusters
plt.scatter(data_reduced[:, 0], data_reduced[:, 1], c=clusters, cmap='viridis', marker='o')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clustered Data in 2D')
plt.show()

## Data Analysis

In [ ]:

# Set the outlier threshold
Q1 = np.percentile(dataset_normalized, 25, axis=0)
Q3 = np.percentile(dataset_normalized, 75, axis=0)
IQR = Q3 - Q1
step = 1.5 * IQR

# Check for outliers
outliers = (dataset_normalized < (Q1 - step)) | (dataset_normalized > (Q3 + step))
outliers_row = np.any(outliers, axis=1)
outlier_indices = np.where(outliers_row)

# Calculate the percentage of outliers
outlier_count = np.sum(outliers_row)
total_points = dataset_normalized.shape[0]
outlier_percentage = f'{round((outlier_count / total_points) * 100, 3)}%'

# Plot the results
plt.figure(figsize=(12, 8))
colors = ['red' if i in outlier_indices[0] else 'blue' for i in range(dataset_normalized.shape[0])]
plt.scatter(dataset_normalized[:, 0], dataset_normalized[:, 1], c=colors, alpha=0.5)
plt.xlabel('Normalized Red')
plt.ylabel('Normalized Green')
plt.title('Outliers in Normalized RGB Data (Red vs. Green)')
plt.grid(True)
plt.show()

# Print indices of outliers
print("Indices of outlier data points:", outlier_indices[0], outlier_percentage)


## Modeling 

In [ ]:
# Clean dataset from outliers
dataset_cleaned = dataset[~outliers_row]

# Set the outlier threshold for 3 bands
# features = dataset[['B2', 'B3', 'B4']]
# label = dataset['class']

# Set the outlier threshold for 6 bands
features = dataset[['B2', 'B3', 'B4', 'B5', 'B6', 'B7']]
label = dataset['class']

# features = dataset_cleaned[['B2', 'B3', 'B4']]
# label = dataset_cleaned['class']

X_train, X_test, y_train, y_test = train_test_split(
    features, label, test_size=0.2, random_state=42
)

### Random Forest

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", sk.metrics.accuracy_score(y_test, y_pred))
print("Prectats import hmeanision:", sk.metrics.precision_score(y_test, y_pred, average='macro'))
print("Recall:", sk.metrics.recall_score(y_test, y_pred, average='macro'))
print("F1 Score:", sk.metrics.f1_score(y_test, y_pred, average='macro'))

### Support Vector Machine

In [ ]:
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", sk.metrics.accuracy_score(y_test, y_pred))
print("Prectats import hmeanision:", sk.metrics.precision_score(y_test, y_pred, average='macro'))
print("Recall:", sk.metrics.recall_score(y_test, y_pred, average='macro'))
print("F1 Score:", sk.metrics.f1_score(y_test, y_pred, average='macro'))

## Cofunsion Matrix

In [ ]:
from matplotlib import pyplot as plt

disp = sk.metrics.confusion_matrix(y_test, y_pred, labels=[0, 1])
plt.figure(figsize=(10, 7))
plt.imshow(disp, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()

classes = [0, 1] 
tick_marks = np.arange(len(classes))

plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

plt.ylabel('True label')
plt.xlabel('Predicted label')

for i in range(disp.shape[0]):
    for j in range(disp.shape[1]):
        plt.text(j, i, disp[i, j],
                 horizontalalignment="center",
                 color="white" if disp[i, j] > disp.max() / 2. else "black")


plt.show()

In [ ]:
place_classified = image.select(['B2', 'B3', 'B4']).classify(clf)
vis_params = {
    'min': 0,
    'max': 2000,
    'palette': ['blue', 'green', 'red'],
    'gamma': gamma
}
map.addLayer(place_classified, vis_params=vis_params, name='Classified');